## Computational Homework 4

In this assignment, we will show that we have (essentially) all the tools to reproduce the output of a linear regression analysis from commonly used statistical software.

For this, we will use the `canadian_wages` dataset, which can be downloaded from [this link](https://drive.google.com/drive/folders/1OkXMcFo0urN0kSQYH4d75I4V3pnSpV6H?usp=sharing).

In [1]:
import pandas as pd

data = pd.read_csv("canadian_wages.csv")
data.head()

,Unnamed: 0,age,hourly_wages,education_years
0,0,40.0,10.56,15.0
1,1,19.0,11.00,13.0
2,2,46.0,17.76,14.0
3,3,50.0,14.00,16.0
4,4,31.0,8.20,15.0


The goal of this dataset is to predict `hourly_wages` using `age` and `education_years`. We'll use the `statsmodels` package in Python as our point of comparison to run a least squares regression (though this will mostly match the outputs you'd get from e.g. the `lm` function in R). If you don't have `statsmodels` installed already, you can do so by uncommenting the following line.

In [2]:
#!pip install statsmodels

Before running a regression, we need to extract the data into numpy arrays, which we do in the following cell.

In [3]:
import numpy as np

y = data["hourly_wages"].to_numpy()
X = data[["age", "education_years"]].to_numpy()
ones = np.ones(X.shape[0]).reshape(-1,1)
X = np.hstack([ones, X])
X.shape, y.shape

((3997, 3), (3997,))

Next, we can use `statsmodels` to run a linear regression analysis using the following.

In [4]:
import statsmodels.api as sm

res = sm.OLS(y, X).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.259
Model:                            OLS   Adj. R-squared:                  0.258
Method:                 Least Squares   F-statistic:                     696.3
Date:                Mon, 10 Oct 2022   Prob (F-statistic):          3.77e-260
Time:                        09:22:56   Log-Likelihood:                -13313.
No. Observations:                3997   AIC:                         2.663e+04
Df Residuals:                    3994   BIC:                         2.665e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -6.2827      0.609    -10.314      0.000      -7.477      -5.088
x1             0.2628      0.009     29.321      0.000       0.245       0.280
x2             0.9170      0.035     25.881      0.000       0.848       0.986
==============================================================================
Omnibus:                      571.348   Durbin-Watson:                   1.961
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1114.699
Skew:                           0.888   Prob(JB):                    8.83e-243
Kurtosis:                       4.882   Cond. No.                         233.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

As we see, this output returns a variety of things, though we will focus on the following components:

1. The fitted coefficients, along with their standard errors and the associated $t$ statistics/p-values for each.
2. The $R^2$ and adjusted $R^2$, which capture the fraction of variance in `hourly_wage` which is explained by the features `age` and `education_years`.
3. The $F$ statistic for testing the null hypothesis $\beta_1=\beta_2 = 0$.

In this assignment, we will write our own code to reproduce these results. Note that you will very likely find code in the online book useful for this.

### Problem 1.

Perform the following steps.

1. Fit the regression model 

$$
\text{hourly_wage}_i = \beta_0 + \beta_1 \text{age}_i + \beta_2 \text{education_years}_i + \varepsilon_i
$$

To find the coefficients $\hat{\beta} = (\hat{\beta}_0,\hat{\beta}_1, \hat{\beta}_2) = (X^\top X)^{-1}X^\top y$. Verify that these match the estimates in the table above.

2. Compute the estimated variance 

$$
\hat{\sigma}^2 = \frac{1}{n-3}\|y - X\hat{\beta}\|_2^2.
$$

Use this to compute the standard deviation of each of the three coefficients, given by $\tau_j = \sqrt{\hat{\sigma}^2(X^\top X)^{-1}_{jj}}$. Verify that these match the estimates in the table above.

3. For each of the coefficients, compute the $t$ statistic $\hat{t}_j = \hat{\beta}_j/\tau_j$. Compute the $p$-value of this statistic under the $t(n-3)$ distribution, and verify that this (and the statistic) match the estimates in the table above.

In [5]:
#1

bhat = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)
print('bhat: ', bhat)
#Verified: these match the estimates in the table above


#2

vhat = 1/(X.shape[0]-3) * np.sum((y - np.dot(X, bhat))**2)
tau = np.sqrt(vhat * np.linalg.inv(np.dot(X.T, X)))
print('tau: ', tau[0,0], tau[1,1], tau[2,2])
#Verified: these match the estimates in the table above


#3

print('t statistics: ', (bhat / tau)[0,0], (bhat / tau)[1,1], (bhat / tau)[2,2])

from scipy.stats import t
t_dist = t(df = (X.shape[0] - 3))
print('p-value: ', 2 * (1 - t_dist.cdf(abs((bhat / tau)[0,0]))), 2 * (1 - t_dist.cdf(abs((bhat / tau)[1,1]))), 2 * (1 - t_dist.cdf(abs((bhat / tau)[2,2]))))
#Verified: these match the estimates in the table above

bhat:  [-6.28268002  0.26279167  0.91697245]
tau:  0.6091694769049752 0.008962647828520423 0.03543095701236729
t statistics:  -10.313517429443625 29.320762427023418 25.88054431085585
p-value:  0.0 0.0 0.0


### Problem 2.

For the regression model fitted in Problem 1, compute the RSS, TSS and RegSS. Using these, compute

$$
R^2 = \text{RegSS}/\text{TSS}
$$

and

$$
R^2_{adj} = 1 - \frac{(1-\text{RegSS}/\text{TSS})(n-1)}{n-3}.
$$

Verify that these match the values for $R^2$ and adjusted $R^2$ in the table from `statsmodels` above.

In [8]:
y_bar = np.mean(y)
y_hat = np.dot(X, bhat)
TSS = np.sum((y-y_bar)**2)
RSS = np.sum((y-y_hat)**2)
RegSS = TSS-RSS
R2 = RegSS/TSS
R2_adj = 1-(1-R2)*(X.shape[0]-1)/(X.shape[0]-X.shape[1])
print('R2: ', R2)
print('R2_adj: ', R2_adj)

#Verified: these match the estimates in the table above

R2:  0.2585300110315174
R2_adj:  0.25815871909913457


### Problem 3.

Again using the regression model fit in Problem 1, compute the $F$-statistic

$$
\hat{F} = \frac{\|\hat{y}_i - \bar{y}\|_2^2 / 2}{\|\hat{y}_i - y_i\|_2^2/(n-3)}
$$

and the associated $p$-value under the $F(2, n-3)$ distribution. Verify that these too match the results output from `statsmodels`.

In [12]:
num = np.sum((y_hat-y_bar)**2)/2
den = np.sum((y_hat-y)**2)/(X.shape[0]-3)
print('F-statistic: ', num/den)

from scipy.stats import f
print('p value: ', round(1 - f.cdf(num/den, 2, X.shape[0] - 3), 4))
print('Prob (F-statistic): ', f.pdf(num/den, 2, X.shape[0] - 3))

#Verified: these match the estimates in the table above

F-statistic:  696.2984877488853
p value:  0.0
Prob (F-statistic):  2.798775781432101e-260
